## SAS program to transpose data sets

```
***********************************************
*   Create a small data set to demonstrate    *
*   the use of proc transpose                 *
***********************************************;
data data1;
  input category $ x y $;
  datalines;
  A 1 y1
  A 2 y2
  B 1 y3
  B 2 y4
  B 3 y5
  C 1 y6
  C 2 y7
  C 3 y8
  C 4 y9 
  ;
  
***********************************************
*   Transpose the data to wider format        *
***********************************************;
proc transpose data=data1 out=data2(drop=_name_) prefix=col;
	by category;
	id x;
	var y;
run;


***********************************************
*   Transpose the data back to longer format  *
***********************************************;
proc transpose data=data2 out=data3;
	by category;
	var col:;
run;

proc sql;
	create table data4 as
	select category,
	       input(substr(_name_,4),8.) as x,
	       col1 as y
	from data3
	where col1 ne "";
quit;
```

# Python code to produce the same data sets

In [1]:
# Import packages/libraries
import pandas as pd

In [2]:
# create the data frame that was used in the SAS program above
data1 = pd.DataFrame({'category':['A','A','B','B','B','C','C','C','C'],
                       'x':[1,2,1,2,3,1,2,3,4],
                       'y':['y'+str(i) for i in range(1,10)]})

data1

,category,x,y
0,A,1,y1
1,A,2,y2
2,B,1,y3
3,B,2,y4
4,B,3,y5
5,C,1,y6
6,C,2,y7
7,C,3,y8
8,C,4,y9


In [3]:
# Transpose the data frame to wide format using the pivot_table method
data2 = data1.pivot_table(index='category',
                          columns='x', 
                          values='y',
                          aggfunc=lambda v:v).add_prefix('col')
data2

x,col1,col2,col3,col4
category,,,,
A,y1,y2,NaN,NaN
B,y3,y4,y5,NaN
C,y6,y7,y8,y9


In [4]:
# Use the stack method to transpose back to the long format
data3 = data2.stack().reset_index().dropna().rename(columns={0:'y'})
data3['x'] = data3['x'].str.replace("col", "").astype('int64')
data3

,category,x,y
0,A,1,y1
1,A,2,y2
2,B,1,y3
3,B,2,y4
4,B,3,y5
5,C,1,y6
6,C,2,y7
7,C,3,y8
8,C,4,y9
